In [1]:
import torch
import random
import numpy as np
import torchsummary

In [2]:
input_data = torch.load('input_data.pt').cuda()
input_data_check = torch.load('input_data_check.pt').cuda()
output_data = torch.load('output_data.pt').cuda()

input_data = input_data.to(torch.float32)#.view(-1, 12, 8, 8)
output_data = output_data.to(torch.float32)

print(input_data.shape)
print(output_data.shape)
centipawn_std = torch.std(output_data)
output_data = output_data / centipawn_std

input_data = input_data.reshape(input_data.shape[0], -1)
input_data_check = input_data_check.reshape(input_data_check.shape[0], -1)
print(input_data.shape)

n_data = input_data.shape[0]


torch.Size([95013, 2, 6, 8, 8])
torch.Size([95013])
torch.Size([95013, 768])


In [3]:
centipawn_std

tensor(644.9344, device='cuda:0')

In [4]:
print(torch.mean(output_data))
print(torch.std(output_data))
print(torch.min(output_data))
print(torch.max(output_data))

tensor(-0.0301, device='cuda:0')
tensor(1.0000, device='cuda:0')
tensor(-14.8930, device='cuda:0')
tensor(14.8930, device='cuda:0')


In [5]:
model = torch.nn.Sequential(torch.nn.Linear(768, 1)).cuda()
model = torch.nn.Sequential(torch.nn.Linear(768, 10),torch.nn.ReLU(),torch.nn.Linear(10, 1)).cuda()
#model = torch.nn.Sequential(torch.nn.Linear(768, 10),torch.nn.ReLU(),torch.nn.Linear(10, 10),torch.nn.ReLU(),torch.nn.Linear(10, 1))
# model = torch.nn.Sequential(torch.nn.Conv2d(12, 16, 3,padding='same'),torch.nn.ReLU(),torch.nn.Conv2d(16, 16, 3,stride=5),torch.nn.ReLU(),torch.nn.Flatten(), torch.nn.Linear(64, 16),torch.nn.ReLU(), torch.nn.Linear(16, 1)).cuda()

# torchsummary.summary(model, (12, 8, 8))

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [6]:
model = model.to(torch.float32)
for i in range(20000):
    loss = torch.mean(torch.square(model(input_data)[:,0]-output_data))
    loss.backward()
    optimizer.step()
    model.zero_grad()
    if i%1000 == 0:     
        print(loss.item())

1.0117806196212769
0.2703206241130829
0.20132943987846375
0.17992885410785675
0.17029914259910583
0.1643807590007782
0.160278782248497
0.15751144289970398
0.1554950624704361
0.1538955271244049
0.15269945561885834
0.15156467258930206
0.15073274075984955
0.150009885430336
0.14941875636577606
0.14878053963184357
0.14814381301403046
0.14776568114757538
0.1473497450351715
0.14703112840652466


In [7]:
print("checksum: ", (model(input_data_check)-model.state_dict()["2.bias"]).item())

checksum:  0.5304069519042969


In [8]:
model.state_dict().keys()

params = []

for layer in range(2):
    w_key = str(layer*2)+'.weight'
    b_key = str(layer*2)+'.bias'
    W = torch.cat((model.state_dict()[w_key].T, model.state_dict()[b_key].reshape(1, -1)), dim=0)
    params += W.T.reshape(-1).tolist()

with open("weights.txt", "w") as text_file:
    for i,p in enumerate(params):
        if i%12 == 0:
            text_file.write('\n')
        text_file.write("{:.4f}, ".format(p))

In [9]:
# batch_idx = range(0,15)#random.sample(range(1,n_data),10)
# print(torch.cat([model(input_data[batch_idx,:])[:,0].view(-1,1),output_data[batch_idx].view(-1,1)] , dim=1)*centipawn_std)

In [10]:
# model = model.cpu()

# model_int8 = torch.ao.quantization.quantize_dynamic(
#     model,  # the original model
#     {torch.nn.Linear},  # a set of layers to dynamically quantize
#     dtype=torch.qint8)

In [11]:
# loss = torch.mean(torch.square(model(input_data.cpu())[:,0]-output_data.cpu()))
# loss_int8 = torch.mean(torch.square(model_int8(input_data.cpu())[:,0]-output_data.cpu()))

# print(loss.item(), loss_int8.item())

In [12]:
# model.state_dict()['0.weight'][0].shape

In [13]:
# torch.mean(torch.square(input_data.cpu() @ model.state_dict()['0.weight'][0] + model.state_dict()['0.bias'][0] - output_data.cpu()))

In [14]:

# # print(list(model.parameters()))
# w = model_int8.state_dict()['0._packed_params._packed_params'][0][0]

# print(w)

In [15]:
# torch.mean(torch.square(input_data.cpu() @ model_int8.state_dict()['0._packed_params._packed_params'][0][0]))# + model_int8.state_dict()['0._packed_params._packed_params'][1] - output_data.cpu()))